# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
twits_total = len(twits['data'])
twits_total

1548010

### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]

# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http\S*', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[a-z0-9]{1,6}', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@[a-z0-9@._]+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z]+', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token, pos='v') for token in tokens if len(token) > 1]  # root verbs
    tokens = [wnl.lemmatize(token, pos='n') for token in tokens]  # root nouns

    assert type(tokens) == list, 'Tokens should be list'
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement
tokenized = [preprocess(message) for message in messages]

In [7]:
len(tokenized)

1548010

In [8]:
tokenized

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'of',
  'hold',
  'set',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'be',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'the', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention'],
 ['ok',
  'good',
  'we',
  're',
  'not',
  'drop',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'in',
  'month',
  'back',
  'to',
  'if',
  'bo',
  'then',
  'bingo',
  'what',
  'be',
  'the',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'be', 'at', 'and', 'short', 

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
bow = Counter([word for message in tokenized for word in message])

In [10]:
sorted(bow, key=bow.get, reverse=True)

['be',
 'the',
 'to',
 'for',
 'on',
 'of',
 'and',
 'in',
 'this',
 'it',
 'at',
 'will',
 'up',
 'buy',
 'report',
 'go',
 'have',
 'you',
 'short',
 'that',
 'what',
 'today',
 'stock',
 'get',
 'here',
 'just',
 'trade',
 'call',
 'sell',
 'down',
 'with',
 'not',
 'day',
 'we',
 'from',
 'all',
 'do',
 'if',
 'now',
 'out',
 'they',
 'earn',
 'more',
 'market',
 'by',
 'but',
 'estimize',
 'like',
 'my',
 'see',
 'look',
 'analyst',
 'share',
 'next',
 'new',
 'a',
 'so',
 'can',
 'good',
 'week',
 'volume',
 'back',
 'expect',
 'no',
 'time',
 'quot',
 'price',
 'hold',
 'come',
 'last',
 'long',
 'put',
 'or',
 'tomorrow',
 'close',
 'growth',
 'amp',
 'still',
 'after',
 'year',
 'move',
 'make',
 'bear',
 'revenue',
 'eps',
 'over',
 'think',
 'amc',
 'bmo',
 'lol',
 'some',
 'take',
 'when',
 'say',
 'than',
 'high',
 'er',
 'big',
 'about',
 'one',
 'let',
 'bull',
 'sale',
 'there',
 'me',
 'before',
 'your',
 'higher',
 'don',
 'bullish',
 'break',
 'money',
 'how',
 'thei

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [11]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count / twits_total for word, count in bow.items()}
print(f'vocabulary: {len(freqs)}')

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word, _ in bow.most_common(high_cutoff)]
print(K_most_common)

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(f'filtered words: {len(filtered_words)}') 

vocabulary: 91072
['be', 'the', 'to', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'buy', 'report', 'go', 'have', 'you', 'short', 'that']
filtered words: 27008


In [12]:
freqs['the']

0.2576268887151892

In [13]:
'the' not in K_most_common

False

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [14]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: index for index, word in enumerate(filtered_words)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {index: word for word, index in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

In [15]:
len(filtered)

1548010

In [16]:
filtered

[['great', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'hold',
  'set',
  'target',
  'price',
  'usd',
  'our',
  'own',
  'verdict'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'reason', 'pay', 'more', 'attention'],
 ['ok', 'good', 'we', 're', 'not', 'drop', 'price', 'over', 'weekend', 'lol'],
 ['daily', 'chart', 'we', 'need', 'get', 'back', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'month',
  'back',
  'if',
  'bo',
  'then',
  'bingo',
  'what',
  'odds'],
 ['strong'],
 ['ratio', 'float', 'via'],
 ['price',
  'squeeze',
  'perfect',
  'place',
  'an',
  'option',
  'straddle',
  'near',
  'support',
  'trend'],
 ['start',
  'new',
  'monday',
  'expect',
  'strong',
  'volume',
  'across',


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [17]:
n_neutral = sum(1 for each in sentiments if each == 2)
print(n_neutral)

n_examples = len(sentiments)
print(n_examples)

keep_prob = (n_examples - n_neutral) / 4 / n_neutral  # keep amount of neutral equally the same as other 4 kinds
print(keep_prob)

701597
1548010
0.3016022730997995


In [18]:
balanced = {'messages': [], 'sentiments':[]}

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [19]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
print(n_neutral)

n_examples = len(balanced['sentiments'])
print(n_examples)

print(n_neutral / n_examples)

200192
1027522
0.19482989172007997


Finally let's convert our tokens into integer ids which we can pass to the network.

In [20]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [21]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup LSTM layer
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
        
        # Setup dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Setup fully connected linear (dense) layer
        self.fc = nn.Linear(lstm_size, output_size)
        
        # Setup softmax layer
        self.lsoftmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes lstm_layers x batch_size x lstm_size,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
    
        hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                        weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
            
        return hidden_state

    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement
        #batch_size = nn_input.size(0)  # since batch_first param is set to True
        
        # Pass thru embedding layer
        embeds = self.embedding(nn_input)
        #print(f'embeds shape: {embeds.shape}')
        
        # Pass thru LSTM layer
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        #print(f'lstm_out shape: {lstm_out.shape}')
        #print(f'hidden_state [0] shape: {hidden_state[0].shape}')
        
        # Stack up LSTM outputs
        #lstm_stack = lstm_out.contiguous().view(-1, self.lstm_size)
        
        # See this for reasoning: https://knowledge.udacity.com/questions/30112
        lstm_stack = lstm_out[-1, :, :]
        #print(f'lstm_stack shape: {lstm_stack.shape}')
        
        # Pass thru dropout layer
        out = self.dropout(lstm_stack)
        
        # Pass thru dense layer
        out = self.fc(out)
        #print(f'out shape: {out.shape}')
        
        # Pass thru log softmax layer
        log_ps = self.lsoftmax(out)
        #print(f'log_ps shape: {log_ps.shape}')
        
        # Reshape to be batch_size first
        #log_ps = log_ps.view(batch_size, -1)
        
        # Get last batch of output
        #log_ps = log_ps[:, -1]
        
        return log_ps, hidden_state

### View Model

In [22]:
model = TextClassifier(len(vocab), 10, 6, 5, lstm_layers=2, dropout=0.1)
model.embedding.weight.data.uniform_(-1, 1)

tensor([[-3.1716e-01,  5.0206e-01, -5.4392e-01,  ..., -4.5404e-01,
          6.1463e-01,  2.7327e-01],
        [-2.9358e-01,  2.5224e-01,  3.3548e-01,  ...,  8.9128e-01,
         -3.0343e-01,  7.0082e-01],
        [ 5.8744e-01, -2.2934e-01,  3.5669e-01,  ..., -3.9964e-02,
         -5.0694e-01,  1.2411e-01],
        ...,
        [ 8.8982e-01,  2.3064e-01,  3.5473e-01,  ..., -5.1934e-01,
          2.3726e-01,  1.1829e-01],
        [-1.7575e-02, -2.0513e-02, -2.4975e-01,  ...,  2.9063e-01,
          8.0811e-01, -3.9593e-01],
        [-4.4865e-01,  6.5903e-01, -9.4928e-01,  ...,  5.0421e-01,
          6.6882e-01, -1.1751e-01]])

In [23]:
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
print(input.shape)

hidden = model.init_hidden(4)
print(hidden[0].shape)

torch.Size([5, 4])
torch.Size([2, 4, 6])


In [24]:
logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.7875, -1.7123, -1.8253, -1.3472, -1.4652],
        [-1.7819, -1.6816, -1.8029, -1.3669, -1.4877],
        [-1.7893, -1.6754, -1.7899, -1.3805, -1.4816],
        [-1.7866, -1.6800, -1.8032, -1.3711, -1.4805]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [25]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for i in range(0, total_sequences, batch_size):
        batch_messages = messages[i: i + batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[i: i + len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [26]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

split_frac = 0.8
split_pos = int(len(token_ids) * split_frac)

train_features, valid_features = token_ids[:split_pos], token_ids[split_pos:]
train_labels, valid_labels = sentiments[:split_pos], sentiments[split_pos:]

In [27]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))

model = TextClassifier(len(vocab) + 1, 200, 128, 5, dropout=0.0)
hidden = model.init_hidden(64)

logps, hidden = model.forward(text_batch, hidden)
print(logps)

tensor([[-1.6004, -1.5669, -1.6269, -1.6830, -1.5744],
        [-1.5642, -1.6736, -1.5959, -1.6640, -1.5556],
        [-1.7566, -1.6724, -1.5018, -1.4541, -1.6971],
        [-1.6851, -1.4128, -1.5988, -1.7149, -1.6660],
        [-1.5585, -1.6382, -1.5464, -1.6838, -1.6269],
        [-1.5502, -1.7532, -1.5666, -1.6617, -1.5324],
        [-1.6874, -1.7706, -1.5150, -1.5534, -1.5444],
        [-1.5785, -1.7146, -1.4522, -1.7933, -1.5454],
        [-1.6356, -1.6325, -1.3573, -1.8646, -1.6225],
        [-1.4595, -1.6492, -1.5522, -1.8515, -1.5765],
        [-1.6920, -1.6064, -1.4783, -1.5847, -1.7026],
        [-1.7321, -1.4848, -1.6384, -1.7005, -1.5157],
        [-1.7989, -1.5761, -1.4440, -1.7160, -1.5514],
        [-1.5673, -1.6475, -1.5467, -1.6808, -1.6109],
        [-1.8300, -1.5340, -1.4874, -1.6794, -1.5537],
        [-1.5647, -1.6749, -1.5950, -1.6593, -1.5590],
        [-1.8523, -1.4050, -1.5894, -1.7166, -1.5417],
        [-1.5150, -1.7917, -1.4923, -1.7571, -1.5320],
        [-

### Training
It's time to train the neural network!

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab) + 1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)

model.to(device)

TextClassifier(
  (embedding): Embedding(27009, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (lsoftmax): LogSoftmax()
)

In [29]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

# Loss and optimazation
learning_rate = 0.001

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training parameters
batch_size = 1024
sequence_length = 100

epochs = 3
clip = 5  # gradient clipping

# Printing parameters
print_every = 100

# Turn on training mode
model.train()

# Epoch loop
for e in range(epochs):
    steps = 0
    print('Starting epoch {}/{}'.format(e + 1, epochs))
    
    # Batch loop
    for text_batch, labels in dataloader(train_features, train_labels, sequence_length=sequence_length, batch_size=batch_size, shuffle=True):
        # TODO Implement: Train Model
        steps += 1
        
        # Initialize hidden state
        hidden = model.init_hidden(labels.shape[0])  # at the last iteration of the epoch, rows of the batch will be left-over
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # Zero accumulated gradients
        model.zero_grad()
        
        # Get the output from the model
        logps, hidden = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(logps.squeeze(), labels)
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        # Loss stats
        accuracy = 0
        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            valid_losses = []
            
            for text_batch, labels in dataloader(valid_features, valid_labels, sequence_length=sequence_length, batch_size=batch_size, shuffle=True):
                valid_hidden = model.init_hidden(labels.shape[0])  # at the last iteration of the epoch, rows of the batch will be left-over

                text_batch, labels = text_batch.to(device), labels.to(device)
                for each in valid_hidden:
                    each.to(device)

                logps, valid_hidden = model(text_batch, valid_hidden)
                valid_loss = criterion(logps.squeeze(), labels)

                valid_losses.append(valid_loss.item())
                
                top_valid, top_class = torch.exp(logps).topk(1)
                accuracy += torch.sum(top_class.squeeze() == labels)
                
            print('Epoch: {}/{}...'.format(e + 1, epochs),
                  'Step: {}...'.format(steps),
                  'Loss: {:.6f}...'.format(loss.item()),
                  'Valid Loss: {:.6f}...'.format(np.mean(valid_losses)),
                  'Valid Accuracy: {:.2f}%'.format(100 * accuracy / len(valid_labels)))
            
            model.train()

Starting epoch 1/3
Epoch: 1/3... Step: 100... Loss: 1.016886... Valid Loss: 1.072540... Valid Accuracy: 56.00%
Epoch: 1/3... Step: 200... Loss: 1.020249... Valid Loss: 0.984152... Valid Accuracy: 60.00%
Epoch: 1/3... Step: 300... Loss: 0.924087... Valid Loss: 0.960820... Valid Accuracy: 61.00%
Epoch: 1/3... Step: 400... Loss: 0.892796... Valid Loss: 0.930793... Valid Accuracy: 63.00%
Epoch: 1/3... Step: 500... Loss: 0.922640... Valid Loss: 0.922122... Valid Accuracy: 63.00%
Epoch: 1/3... Step: 600... Loss: 0.852484... Valid Loss: 0.929649... Valid Accuracy: 63.00%
Epoch: 1/3... Step: 700... Loss: 0.872473... Valid Loss: 0.903814... Valid Accuracy: 64.00%
Epoch: 1/3... Step: 800... Loss: 0.880454... Valid Loss: 0.911815... Valid Accuracy: 64.00%
Starting epoch 2/3
Epoch: 2/3... Step: 100... Loss: 0.868457... Valid Loss: 0.919101... Valid Accuracy: 64.00%
Epoch: 2/3... Step: 200... Loss: 0.773636... Valid Loss: 0.898544... Valid Accuracy: 65.00%
Epoch: 2/3... Step: 300... Loss: 0.807926.

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [30]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab]

    # Convert words to ids
    tokens =  [vocab[token] for token in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.LongTensor(tokens).view(-1, 1)))

    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)

    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [31]:
text = "Google is working on self driving cars, I'm bullish on $goog"

model.eval()
model.to("cpu")

predict(text, model, vocab)

tensor([[ 0.0002,  0.0181,  0.0069,  0.5470,  0.4278]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

The prediction of the model gave highest positive sentiment on the twit 0.5470 (54%). Models always have uncertainty, in this case, this model has 46% uncertainty.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [32]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [33]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [34]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [35]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1715,  0.2431,  0.1807,  0.1782,  0.2266]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.